# Process land cover data

NB: for plotting we want to be able to highlight the areas of permanent water.  
To do this, I export a raster where freshwater has a value of 11 and all else np.nan.  
In QGIS: select Raster -> Conversion -> Polygonise (raster-vector) -> (and select this layer)

In [112]:
catchment_name  = 'WykeBeck'
model_directory = '../../../FloodModelling/{}Models/'.format(catchment_name)

# Something in setting crop to to true for Wyke Beck doesn't work (the land cover and results files have different
# values for out_meta so end up different sizes)
if catchment_name =='WykeBeck':
    crop_or_not = False
elif catchment_name == 'LinDyke':
    crop_or_not = True    

In [113]:
import sys
import numpy.ma as ma
import rasterio
from shapely.geometry import box
import geopandas as gpd
from pyproj import CRS
from rasterio.mask import mask
import numpy as np

# Define whether to filter out values <0.1
remove_little_values = True

sys.path.append("../ProcessModelResults/")
from my_functions import *

# Specify strings relating to catchment
if catchment_name == 'LinDyke':
    minx, miny, maxx, maxy = 437000,  426500,  445500, 434300
elif catchment_name == 'WykeBeck':
    minx, miny, maxx, maxy = 430004,  429978, 438660, 440996 
    
# Create a bounding box (this is used in preparing the rasters)
bbox = box(minx, miny, maxx, maxy)    

### Read in main catchment boundary and add to boundaries dictionary

In [114]:
if catchment_name == 'LinDyke':
    catchment_gdf = gpd.read_file(model_directory + 'Model_IdealisedProfiles/CatchmentLinDyke_exported.shp'.format(catchment_name))
elif catchment_name == 'WykeBeck':
    catchment_gdf = gpd.read_file(model_directory + 'Model_IdealisedProfiles/WykeBeckCatchment.shp'.format(catchment_name))

### Read in the land cover file, clip it, update some parameters, save to landcover directory 

In [115]:
if catchment_name == 'LinDyke':
    landcover_clipped, out_meta  = open_and_clip_to_catchment(model_directory + f"LandCoverData/LandCover_resized.tif", 
                                                              catchment_gdf, crop_or_not=crop_or_not)
elif catchment_name == 'WykeBeck':
    landcover_clipped, out_meta  = open_and_clip_to_catchment(model_directory + f"LandCoverData/LandCover_clipped_MA_Download_resampled2m.tif", 
                                                              catchment_gdf, crop_or_not=crop_or_not)


fp_to_save= model_directory + 'LandCoverData/LandCover_clipped.tif'
save_array_as_raster(landcover_clipped, fp_to_save, out_meta)

In [116]:
classes_dict_lindyke = {'SubUrban':1, 'Arable':2, 'Deciduous woodland':3, 'Freshwater':4, 'Improved grassland':5, 'Urban':6, 
                  'Neutral grassland':7, 'Calcareous grassland':8, 'Heather grassland':9}

classes_dict_wykebeck = {'Deciduous woodland':1, 'Coniferous woodland':2, 'Arable':3, 'Improve grassland ':4, 
                         'Calcareous grassland':6, 'Heather':9,'Heather grassland':10,
                         'Inland Rock':12, 'Freshwater':14, 'Urban':20, 'SubUrban':21}

classes_dicts = {'LinDyke':classes_dict_lindyke, 'WykeBeck':classes_dict_wykebeck}

In [117]:
# landcover_clipped= np.where(landcover_clipped==6, 5, landcover_clipped)
# landcover_clipped= np.where(landcover_clipped==9, 6, landcover_clipped)
# landcover_clipped= np.where(landcover_clipped==10, 7, landcover_clipped)
# landcover_clipped= np.where(landcover_clipped==12, 8, landcover_clipped)
# landcover_clipped= np.where(landcover_clipped==14, 9, landcover_clipped)
# landcover_clipped= np.where(landcover_clipped==20, 10, landcover_clipped)
# landcover_clipped= np.where(landcover_clipped==21, 11, landcover_clipped)
# landcover_clipped= np.where(landcover_clipped==21, 11, landcover_clipped)
landcover_clipped= np.where(landcover_clipped==255, np.nan, landcover_clipped)

### Not water
Not water is set to 10, water is set to 11

In [118]:
water_class = classes_dicts[catchment_name]['Freshwater']

In [119]:
# Get list of all classes
ls_values = np.unique(landcover_clipped[~np.isnan(landcover_clipped)]).tolist()
# Remove the water class (so it is a list of all classes but this one)
ls_values.remove(water_class)
# Set the value of the water class to 11
landcover_not_water= np.where(landcover_clipped==water_class, 16, landcover_clipped)
# Set all the rest to 10
for i in ls_values:
    landcover_not_water = np.where(landcover_not_water==i, 15, landcover_not_water)
fp_to_save= model_directory + 'LandCoverData/LandCover_notwater_classification.tif'
save_array_as_raster(landcover_not_water, fp_to_save, out_meta)

### Save version with just the water as 11 and everything else as np.nan
This is for turning into a vector outline for plotting where the permanent water areas are 

In [120]:
# Get list of all classes
ls_values = np.unique(landcover_clipped[~np.isnan(landcover_clipped)]).tolist()
# Remove the water class (so it is a list of all classes but this one)
ls_values.remove(water_class)
# Set the value of the water class to 11
landcover_not_water= np.where(landcover_clipped==water_class, 16, landcover_clipped)
# Set all the rest to 10
for i in ls_values:
    landcover_not_water = np.where(landcover_not_water==i, np.nan, landcover_not_water)
fp_to_save= model_directory + 'LandCoverData/LandCover_water_classification.tif'
save_array_as_raster(landcover_not_water, fp_to_save, out_meta)

### Urban and suburban combo
Urban and suburban are set to 10, all the rest set to 11

In [121]:
ls_values = np.unique(landcover_clipped[~np.isnan(landcover_clipped)]).tolist()
ls_values.remove(classes_dicts[catchment_name]['Urban'])
ls_values.remove(classes_dicts[catchment_name]['SubUrban'])

# Convert the 3 to 10 (for water) and the rest to 11 (for non-water).  
landcover_urban_and_suburban = np.where(landcover_clipped==classes_dicts[catchment_name]['Urban'], 15, landcover_clipped)
landcover_urban_and_suburban = np.where(landcover_urban_and_suburban==classes_dicts[catchment_name]['SubUrban'], 15, landcover_urban_and_suburban)

# Convert the rest of the classes to 11
for i in ls_values:
    landcover_urban_and_suburban =  np.where(landcover_urban_and_suburban==i, 16, landcover_urban_and_suburban)

fp_to_save= model_directory + 'LandCoverData/LandCover_urban_and_suburban_classification.tif'
save_array_as_raster(landcover_urban_and_suburban, fp_to_save, out_meta)

## Check sizes

In [122]:
print(landcover_not_water.shape)
print(landcover_urban_and_suburban.shape)

(5509, 4328)
(5509, 4328)
